# [Fe/H] Metallicity Distribution

In [65]:
from multiprocessing import Pool
import os
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic
from scipy.stats import gaussian_kde

In [24]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [25]:
figure_setup()
settings = Settings()

In [26]:
N_BINS: int = 40

In [27]:
def read_data(simulation: str, of: str, to: str,
              tag_in_situ: bool = False) -> tuple:
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=settings.disc_std_circ,
        disc_min_circ=settings.disc_min_circ,
        cold_disc_delta_circ=settings.cold_disc_delta_circ,
        bulge_max_specific_energy=settings.bulge_max_specific_energy)
    s.add_metal_abundance(of, to)
    
    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df[f"[{of}/{to}]"] = s.metal_abundance[
        f"{of}/{to}"][is_real_star & is_main_obj]
    df["RegionTag"] = s.region_tag[is_real_star & is_main_obj]

    if tag_in_situ:
        s.tag_in_situ_stars()
        df["IsInSitu"] = s.is_in_situ[
            is_real_star & is_main_obj].astype("bool")

    return df

In [28]:
simulations = [f"au{i}_or_l4_s127" for i in range(1, 31)]

In [32]:
def plot_distribution_for_galaxy_with_stellar_origin(
    simulation: str, of: str, to: str,
    xlim: tuple, ylim: tuple, xticks: list) -> None:

    df = read_data(simulation=simulation, of=of, to=to, tag_in_situ=True)

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_xticks(xticks)
        ax.set_xlabel(f'[{of}/{to}]')
        ax.set_ylabel(r"$f_\star$")
        ax.grid(True, ls='-', lw=0.25, c="gainsboro")
        ax.label_outer()

    for idx, component in enumerate(settings.components):

        n_stars_comp = (df["RegionTag"] == idx).sum()

        hist, bin_edges = np.histogram(
            a=df[f"[{of}/{to}]"][df["RegionTag"] == idx],
            bins=N_BINS, range=xlim)
        bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
        axs[idx].plot(
            bin_centers, hist / n_stars_comp, zorder=11,
            c=settings.component_colors[component], lw=1)

        hist, _ = np.histogram(
            a=df[f"[{of}/{to}]"][
                (df["RegionTag"] == idx) & (df["IsInSitu"] == 1)],
            bins=N_BINS, range=xlim)
        axs[idx].plot(
            bin_centers, hist / n_stars_comp, zorder=11,
            c='k', lw=0.75, ls=(0, (3, 1.5, 1, 1.5)), label="In-Situ Stars")

        hist, _ = np.histogram(
            a=df[f"[{of}/{to}]"][
                (df["RegionTag"] == idx) & (df["IsInSitu"] == 0)],
            bins=N_BINS, range=xlim)
        axs[idx].plot(
            bin_centers, hist / n_stars_comp, zorder=11,
            c='k', lw=0.75, ls=(0, (5, 1.5)), label="Ex-Situ Stars")

        axs[idx].text(x=0.05, y=0.95, size=8.0, ha="left", va="top",
                    s=settings.component_labels[settings.components[idx]],
                    transform=axs[idx].transAxes)

    axs[3].legend(loc="lower left", framealpha=0, fontsize=6,
                    bbox_to_anchor=(0.009, 0.65))

    axs[0].text(
        x=0.95, y=0.95, size=6.0,
        s=r"$\texttt{" + simulation.upper() + "}$",
        ha='right', va='top', transform=axs[0].transAxes)

    fig.savefig(
        f"../images/metal_abundance_distribution/{of}_{to}/"
        f"with_stellar_origin/{simulation}.pdf")
    plt.close(fig)

In [71]:
def plot_distribution_for_sample(simulations: list,
    of: str, to: str, xlim: tuple, ylim: tuple,
    xticks: list, yticks: list) -> None:

    fig = plt.figure(figsize=(7.4, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_xticks(xticks)
        ax.set_yticks(yticks)
        ax.set_xlabel(f'[{of}/{to}]')
        ax.set_ylabel("PDF")
        ax.grid(True, ls='-', lw=0.25, c="gainsboro")
        ax.label_outer()

    for i, simulation in enumerate(simulations):
        df = read_data(simulation=simulation, of=of, to=to)
        ax = axs.flatten()[i]

        for region_tag in range(4):
            is_region = (df["RegionTag"] == region_tag)
            is_finite = np.isfinite(df[f"[{of}/{to}]"])

            pdf = gaussian_kde(df[f"[{of}/{to}]"][
                is_region & is_finite].to_numpy(
                    dtype=np.float64))
            pdf_x = np.linspace(xlim[0], xlim[1], 100)
            pdf_y = pdf(pdf_x)
            ax.plot(
                pdf_x, pdf_y, lw=1.0,
                color=settings.component_colors[
                    settings.components[region_tag]],
                label=settings.component_labels[
                    settings.components[region_tag]],
                zorder=15)

        ax.text(
            x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + simulation.upper() + "}$",
            ha='right', va='top', transform=ax.transAxes)

        if i == 0:
            ax.legend(loc="upper left", framealpha=0, fontsize=4)

        fig.savefig(
            f"../images/metal_abundance_distribution/{of}_{to}/originals.pdf")
        plt.close(fig)

In [34]:
for simulation in simulations:
    plot_distribution_for_galaxy_with_stellar_origin(
        simulation, of="Fe", to='H', xlim=(-3, 1.5), ylim=(0, 0.2),
        xticks=[-2, -1, 0, 1])

In [77]:
plot_distribution_for_sample(
    simulations, of='O', to="Fe",
    xlim=(0, 0.5), ylim=(0, 20),
    xticks=[0.1, 0.2, 0.3, 0.4], yticks=[0, 5, 10, 15])

In [76]:
plot_distribution_for_sample(
    simulations, of="Fe", to='H',
    xlim=(-2, 2), ylim=(0, 2.5),
    xticks=[-1, 0, 1], yticks=[0, 0.5, 1, 1.5, 2])